In [ ]:
import numpy as np
import pandas as pd
!pip install emoji
import emoji
import tensorflow as tf
import keras.losses

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
data = pd.read_csv('emoji_prediction_data.csv', header = None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [ ]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])

In [ ]:
X = data[0].values
Y = data[1].values
Y

array([4, 3, 3, 1, 2, 1, 4, 3, 4, 1, 3, 3, 2, 2, 4, 3, 2, 3, 3, 1, 3, 2,
       2, 2, 0, 1, 0, 4, 2, 2, 2, 0, 0, 3, 4, 0, 2, 1, 3, 1, 0, 4, 0, 3,
       0, 4, 2, 3, 4, 2, 2, 3, 0, 2, 2, 3, 2, 3, 2, 2, 3, 3, 0, 2, 3, 0,
       2, 0, 0, 2, 3, 2, 4, 1, 3, 3, 0, 0, 3, 2, 0, 3, 0, 2, 2, 4, 2, 2,
       0, 0, 2, 3, 0, 4, 2, 1, 2, 3, 3, 2, 3, 0, 3, 0, 2, 0, 2, 3, 4, 3,
       1, 3, 4, 3, 2, 3, 3, 3, 1, 4, 4, 2, 2, 1, 1, 2, 3, 2, 3, 4, 2, 3,
       0, 2, 0, 0, 4, 3, 4, 2, 3, 2, 3, 4, 2, 1, 2, 4, 3, 1, 3, 2, 3, 2,
       2, 3, 3, 2, 4, 0, 0, 0, 3, 0, 0, 1, 1, 2, 2, 2, 0, 3, 2, 3, 3, 1,
       2, 2, 4, 2, 3, 1, 2])

In [ ]:
file = open('glove.6B.100d.txt', 'r', encoding = 'utf8')
content = file.readlines()
file.close()

# content

In [ ]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

Xtokens = tokenizer.texts_to_sequences(X)

In [ ]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

maxlen = get_maxlen(Xtokens)
print(maxlen)

Xtokens

10


[[103, 104, 3, 6, 105],
 [106, 3, 107],
 [1, 7, 108],
 [109, 4, 35],
 [36, 30],
 [37, 3, 19, 110, 26, 49],
 [1, 111, 112],
 [31, 67, 113],
 [1, 20, 114, 27],
 [115, 68, 38, 69, 26],
 [2, 11, 116, 10, 70],
 [117, 50, 71, 51],
 [36, 39],
 [12, 12, 12, 22, 28, 6, 40],
 [1, 32, 21, 5, 118, 119],
 [120, 11, 2, 121, 41],
 [1, 20, 9, 30],
 [1, 72, 52, 53, 13, 10],
 [4, 122, 3, 123],
 [73, 3, 4, 35],
 [1, 7, 124],
 [12, 12, 12, 54],
 [14, 52, 53],
 [15, 23, 5, 125],
 [126, 3, 127, 1, 21],
 [15, 74, 128, 129, 75],
 [1, 18, 9, 4, 130, 55, 131],
 [29, 2, 24, 132],
 [2, 133, 134, 10, 135],
 [1, 33, 2, 6, 76],
 [1, 24, 19, 136],
 [14, 23, 16, 5, 137],
 [32, 2, 138, 8, 77],
 [2, 139, 4, 140],
 [141, 3, 56, 13, 5, 78],
 [77, 42, 3, 142],
 [43, 6, 79],
 [29, 2, 21, 5, 35],
 [15, 74, 17, 29, 143],
 [15, 80, 26],
 [50, 57, 81, 5, 144, 44, 145],
 [146, 11, 147],
 [15, 3, 5, 36, 148],
 [72, 149, 9, 16, 51],
 [1, 33, 58],
 [27, 3, 34],
 [1, 7, 150, 59],
 [6, 41, 45, 2, 151, 152, 46, 68],
 [29, 2, 20, 9, 15

In [ ]:
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')
Xtrain

array([[103, 104,   3, ...,   0,   0,   0],
       [106,   3, 107, ...,   0,   0,   0],
       [  1,   7, 108, ...,   0,   0,   0],
       ...,
       [ 14,   3,   5, ...,   0,   0,   0],
       [ 14, 310,  26, ...,   0,   0,   0],
       [  1,  24,  22, ...,   0,   0,   0]], dtype=int32)

In [ ]:
Ytrain = to_categorical(Y)
Ytrain

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0.

In [ ]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))

for word, i in word2index.items():
    if word in embeddings:
        embed_vector = embeddings[word]
    else:
        embed_vector = np.random.rand(embed_size)
    embedding_matrix[i] = embed_vector

embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.046539,  0.61966 ,  0.56647 , ..., -0.37616 , -0.032502,
         0.8062  ],
       [-0.49886 ,  0.76602 ,  0.89751 , ..., -0.41179 ,  0.40539 ,
         0.78504 ],
       ...,
       [-0.46263 ,  0.069864,  0.69095 , ..., -0.29174 ,  0.32041 ,
         0.21202 ],
       [ 0.073242,  0.11134 ,  0.62281 , ...,  0.53417 , -0.1646  ,
        -0.27516 ],
       [ 0.29019 ,  0.80497 ,  0.31187 , ..., -0.33603 ,  0.45998 ,
        -0.11278 ]])

In [ ]:
model = Sequential([
    Embedding(input_dim = len(word2index)+1,
              output_dim = embed_size,
              input_length = maxlen,
              weights = [embedding_matrix],
              trainable = False
     ),
    LSTM(units = 16, return_sequences = True),
    #LSTM(units = 10, return_sequences = True),
    LSTM(units = 4),
    Dense(5, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

In [ ]:
#model.predict(np.ones((1, maxlen))).shape

In [ ]:
model.fit(Xtrain, Ytrain, epochs = 100)

Epoch 1/100
6/6 [==============================] - 5s 11ms/step - loss: 1.5946 - accuracy: 0.3279
Epoch 2/100
6/6 [==============================] - 0s 10ms/step - loss: 1.5812 - accuracy: 0.3388
Epoch 3/100
6/6 [==============================] - 0s 14ms/step - loss: 1.5691 - accuracy: 0.3497
Epoch 4/100
6/6 [==============================] - 0s 16ms/step - loss: 1.5562 - accuracy: 0.3825
Epoch 5/100
6/6 [==============================] - 0s 17ms/step - loss: 1.5441 - accuracy: 0.3934
Epoch 6/100
6/6 [==============================] - 0s 20ms/step - loss: 1.5292 - accuracy: 0.3880
Epoch 7/100
6/6 [==============================] - 0s 16ms/step - loss: 1.5143 - accuracy: 0.3825
Epoch 8/100
6/6 [==============================] - 0s 16ms/step - loss: 1.4974 - accuracy: 0.3770
Epoch 9/100
6/6 [==============================] - 0s 15ms/step - loss: 1.4790 - accuracy: 0.3825
Epoch 10/100
6/6 [==============================] - 0s 26ms/step - loss: 1.4613 - accuracy: 0.3880
Epoch 11/100
6/6 [=

In [ ]:
test = ['I want to eat lunch', 'I feel bad', 'I am happy']
#conert these sentences into padded sequenced tokens
test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis =1)
#y_pred #here we get the tokens, but now we will convert them to emojis

for i in range(len(test)):
  print(test[i], label_to_emoji(y_pred[i]))

1/1 [==============================] - 0s 24ms/step
I want to eat lunch 🍽️
I feel bad 😞
I am happy 😃
